<img src="../Data/images/ZumiHeader.png" width=700>

# Applications of KNN: Red Light Green Light

Have you ever played the game "Red Light Green Light"? If you aren't familiar with the game, a referee calls out "red light" or "green light" to players who are trying to race to a finish line. If the referee calls out red light, players must stop and freeze in their places. If the referee calls "green light", players race to the finish line. Whoever arrives first, wins!

In this lesson, you will teach Zumi that green means go and red means stop. This is pretty important for self-driving cars to know because they encounter trafficlights on a daily basis. 



### Step 1: Import Libraries
In this lesson we are using the Zumi library for drive commands, Camera library for training, Screen library for printing text and emotions, and threading for running multiple processes at once.

In [ ]:
from zumi.zumi import Zumi
from zumi.util.camera import Camera
from zumi.util.screen import Screen
from threading import Thread

import sys
sys.path.insert(0,'/home/pi/Zumi_Content/Data/knn-classifier')
from color_classifier import ColorClassifier


zumi = Zumi()
camera = Camera()
screen = Screen()

### Step 2: Gather data

Train Zumi with the green and red color cards. Name your demo ```red_green``` so you can find it again easily and use the following labels and key commands:
* Label: ```green``` Key Command: ```g```
* Label: ```red``` Key Command: ```r```


In [ ]:
try:
    knn = ColorClassifier()
    train = knn.set_values()
    
    if train:
        print("Start gathering data. If you want to stop, press q")
        camera.start_camera()
        cnt = int(input("Type the amount of the pictures that you want to take at once : "))
        while True:
            knn.add_datas(camera, cnt)
            if knn.check_enough_datas(balance=True):
                break

        knn.save_data_set()
        knn.get_accuracy()
finally:
    camera.close()

### Step 3: Test the model
Before we start adding drive commands, let's test the model to check that the data has been trained correctly. Test it with some other objects too to see how reliable it is. If you need to, retrain the data under a different model name.

In [ ]:
try:
    print("Start predicting, press enter to predict!")
    camera.start_camera()
    hsv = input("Type what do you want to use for prediction between <hsv>, ex)h , hv, hsv, v")
    
    while True:
        if input("press enter (or q to exit) : ") == 'q':
            break
        image = camera.capture()
        predict = knn.predict(image, hsv)
        print(predict)
finally:
    camera.close()


### Step 4: Program logic
Now that the model has been tested, it's time to write out the logic with print statements.

When you call ```knn.predict(image)```, it returns the label of its prediction. For example, if you train the model with labels ```blue``` and ```yellow```, some code might look like this:

```
while True:
    if input("Press enter to predict or q to exit.") == "q":
        break
    image = camera.capture()
    predict = knn.predict(image)
    if predict == "blue":
        zumi.forward()
    if predict == "yellow":
        zumi.reverse()
camera.close()
```
Notice that the labels are in quotes. This is because they are **string** data types. Strings are strings of characters, letters, or numbers which are treated as text. These labels must match exactly to your data labels. If you accidentally mistyped "blu" for your label but wrote your code to check if predict is equal to "blue", this code will never happen and you may be wondering what went wrong.


Let's write the program logic for Red Light Green Light with print statements:

In [ ]:
try:
    print("Start predicting, press enter to predict!")
    camera.start_camera()
    hsv = input("Type what do you want to use for prediction between <hsv>, ex)h , hv, hsv, v")
    
    while True:
        if input("press enter (or q to exit) : ") == 'q':
            break
        image = camera.capture()
        predict = knn.predict(image,hsv)
        if predict == "green":
            print("Start driving forward")
        if predict == "red":
            print("Stop")

finally:
    camera.close()

### Step 5: Threading and go_straight()
It may be your intuition to write out the program logic in this way:
```
if predict == "green":
        zumi.forward()
    if predict == "red":
        zumi.stop()
```

However, remember that ```forward()``` drives Zumi for a specified number of seconds and then stops. We want Zumi to stop driving only when she sees a red card. Therefore, we need to use threading. Threading allows us to run multiple processes at the same time. Think of it like weaving two separate functions together so that they appear to be running simultaneously.

When we start a thread, it starts running in the background and allows the rest of the program to continue running. We use a **boolean** to escape from the thread when Zumi sees red. This variable must be declared as **global**, as in it can ben accessed by all functions. Here, we named our variable ```is_green``` and we initially set it to false so that Zumi begins at a stop. We also define out function ```continue_straight```, which will need to be running in the background while ```is_green``` is true. Once Zumi sees red, ```is_green``` becomes false, causing the while loop to break and Zumi to stop.

In [ ]:
global is_green
is_green = False

def continue_straight():
    while is_green:
        zumi.go_straight(20, 0)
    zumi.stop(0)

Now we need to "weave" these codes together with threading.  If Zumi sees green, we need to set the boolean to ```True``` and begin the thread. If Zumi sees red, ```is_green``` is now ```False```.

```
  if predict == "green":
        is_green = True
        drive_thread = Thread(target=continue_straight)
        drive_thread.start()
    if predict == "red":
        is_green = False
        zumi.stop()
```


Putting it altogether,here is the code. Try it out!

In [ ]:
try:
    print("Start predicting, press enter to predict!")
    camera.start_camera()
    hsv = input("Type what do you want to use for prediction between <hsv>, ex)h , hv, hsv, v")
    
    while True:
        if input("press enter (or q to exit) : ") == 'q':
            break
        image = camera.capture()
        predict = knn.predict(image, hsv)
        if predict == "green":
            is_green = True
            drive_thread = Thread(target=continue_straight)
            drive_thread.start()
        if predict == "red":
            is_green = False
            zumi.stop()
finally:
    is_green = False
    zumi.stop()
    camera.close()

If you want, add more colors to red and green and have Zumi do different drive commands based on the color she sees!